### Decision Tree Regressor
#### Problem Statement: Build a model that predicts the Power Engery generated by the powerplant by taking Atmospheric pressure,temperate, vaccum and humidity into consideration
##### Step1: Data Gathering

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')